In [11]:
import pandas as pd
import math
from datetime import datetime

super_dataset = pd.read_csv('../../../data/stage_I/superDataset.csv')
world_dataset = pd.read_csv('../../../data/stage_II/owid-covid-data.csv')

countries = ['Indonesia', 'Pakistan', 'Nigeria', 'Brazil', 'Bangladesh', 'United States']

In [8]:
filtered_dataset = world_dataset[world_dataset.location.isin(countries)]
filtered_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926


In [9]:
us_dataset = filtered_dataset[filtered_dataset.location == 'United States'].fillna(0)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926


In [10]:
us_dataset['date'] = pd.to_datetime(us_dataset.date)
us_dataset['week'] = us_dataset.date.apply(lambda x: str(x.year) + "_" + str(x.weekofyear))
us_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,week
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_8
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9


In [43]:
weekly_data = pd.DataFrame(us_dataset.groupby('week').mean().new_cases.apply(lambda x: math.ceil(x))).reset_index()
# weekly_data

In [44]:
weekly_data['date'] = weekly_data.week.apply(lambda x: datetime.strptime(x + '_1', '%Y_%W_%w'))
weekly_data.sort_values(by="date", inplace=True)
weekly_data.drop('week', inplace=True, axis=1)
weekly_data

,new_cases,date
30,1,2020-01-27
41,1,2020-02-03
46,1,2020-02-10
47,1,2020-02-17
48,1,2020-02-24
49,3,2020-03-02
0,70,2020-03-09
1,385,2020-03-16
2,4527,2020-03-23
3,15574,2020-03-30
